In [4]:
import pandas as pd
import numpy as np
from scipy.stats import gamma, norm

# ============================
# Load Data
# ============================
file = r"D:\climate change\monthly_averages.xlsx"

df = pd.read_excel(file, skiprows=2)   # ✅ Skip Lon/Lat rows
df.rename(columns={df.columns[0]: "Date"}, inplace=True)

df["Date"] = pd.to_datetime(df["Date"])
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month

stations = [c for c in df.columns if c not in ["Date", "Year", "Month"]]

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

# ============================
# SPI Gamma Function
# ============================
def spi_gamma(series):
    data = series.replace([np.inf, -np.inf], np.nan).dropna()

    if len(data) < 3 or (data > 0).sum() < 3:
        return pd.Series([0]*len(series), index=series.index)

    try:
        pos_data = data[data > 0]
        shape, loc, scale = gamma.fit(pos_data, floc=0)
        cdf = gamma.cdf(data, shape, loc=loc, scale=scale)
        spi = norm.ppf(cdf)
        return pd.Series(spi, index=data.index)
    except:
        z = (data - data.mean()) / data.std()
        return pd.Series(z, index=data.index)

# ============================
# Rolling SPI Calculator
# ============================
def compute_spi(df, window):
    spi_df = df[["Year", "Month"]].copy()
    for st in stations:
        print(f"Computing SPI-{window} for {st}...")
        roll_sum = df[st].rolling(window).sum()
        spi_df[st] = spi_gamma(roll_sum)
    return spi_df

SPI_1 = compute_spi(df, 1)
SPI_3 = compute_spi(df, 3)
SPI_6 = compute_spi(df, 6)

SPI_1["SPI_Type"] = "SPI-1"
SPI_3["SPI_Type"] = "SPI-3"
SPI_6["SPI_Type"] = "SPI-6"

SPI_all = pd.concat([SPI_1, SPI_3, SPI_6], ignore_index=True)

# ============================
# Save to Excel
# ============================
output = r"D:\climate change\SPI_output.xlsx"
with pd.ExcelWriter(output) as writer:
    SPI_1.to_excel(writer, sheet_name="SPI-1", index=False)
    SPI_3.to_excel(writer, sheet_name="SPI-3", index=False)
    SPI_6.to_excel(writer, sheet_name="SPI-6", index=False)
    SPI_all.to_excel(writer, sheet_name="ALL_SPI", index=False)

print("✅ SPI finished and saved at:", output)


Computing SPI-1 for 28.04...
Computing SPI-1 for 26.35...
Computing SPI-1 for 30.73...
Computing SPI-1 for 14.71...
Computing SPI-1 for 24.73...
Computing SPI-1 for 25.54...
Computing SPI-1 for 23.31...
Computing SPI-1 for 26.06...
Computing SPI-1 for 30.84...
Computing SPI-1 for 26.58...
Computing SPI-1 for 27.57...
Computing SPI-1 for 23.74...
Computing SPI-1 for 30.16...
Computing SPI-1 for 17.8...
Computing SPI-1 for 25.68...
Computing SPI-1 for 10.45...
Computing SPI-1 for 23.54...
Computing SPI-1 for 22.7...
Computing SPI-1 for 20.51...
Computing SPI-1 for 20.25...
Computing SPI-1 for 34.94...
Computing SPI-1 for 33.64...
Computing SPI-1 for 21.27...
Computing SPI-3 for 28.04...
Computing SPI-3 for 26.35...
Computing SPI-3 for 30.73...
Computing SPI-3 for 14.71...
Computing SPI-3 for 24.73...
Computing SPI-3 for 25.54...
Computing SPI-3 for 23.31...
Computing SPI-3 for 26.06...
Computing SPI-3 for 30.84...
Computing SPI-3 for 26.58...
Computing SPI-3 for 27.57...
Computing SPI-3 